### Introduction

Our team participated in the [Scripts of Tribute AI competition](https://github.com/ScriptsOfTribute), which uses a re-adapted version of the card game *Tales of Tribute* from *The Elder Scrolls Online* to enable AI development. This game belongs to the deck-building, competitive genre, in which players must construct and adapt their decks to overcome their opponent. Each match is played between two players who control decks composed of cards, each providing unique effects or choices. Players manage three resources: coins, prestige, and power and patrons favor.

The game ends under two possible conditions. First, when a player reaches at least 40 prestige points and maintains a lead over the opponent during the subsequent turn; if this cycle continues, the match concludes once a player reaches 80 prestige. Alternatively, the game can also end if a player secures the favor of all patrons.

From the perspective of artificial intelligence, *Scripts of Tribute* presents a complex domain. The game environment is characterized by a large state space, with outcomes that can be non-deterministic due to the random effects of certain actions. Moreover, the game evolves through strategic phases referred to as the early game, mid-game, and end game, each of which emphasizes different priorities. Success therefore requires adaptive strategies that respond both to the evolving game state and to its phase.

Our approach employs a Monte Carlo Tree Search (MCTS) algorithm, which evaluates possible moves until the end of the player’s turn. To guide the search process, we developed heuristic evaluation functions, both manually engineered and automatically optimized through evolutionary algorithms, to assess terminal states within each turn. Additional pruning techniques were applied to reduce the size of the state space. The central component of this design is our heuristic, which we call the **Min-Max Hand Value Rating (MMHVR)**. This heuristic considers multiple game factors: some, such as prestige, power, coins, and patron favors, represent the player’s resources, while others capture deck composition and evaluate the quality and strategic potential of the deck constructed throughout the match.

To evaluate our approach, we compared the performance of our MCTS-based agent against both a random bot and a simpler strategy baseline. The experimental results demonstrate the effectiveness of MMHVR when applied to assess game states within a limited depth-first exploration strategy, as well as its advantages when serving as the evaluation function for MCTS.

All code and this report can be found at the [GitHub repository](https://github.com/VitoBarra/AIF-Bot-ScriptOfTribute).

### Methodologies
The following sections detail the methodologies employed in our approach, including the implementation of the Monte Carlo Tree Search algorithm, the design of the MMHVR heuristic, and the evolutionary optimization process used to refine the heuristic parameters.


#### Monte Carlo Tree Search

In our work, we employ **Monte Carlo Tree Search (MCTS)** as the primary mechanism for move selection, this choice is driven by three reason: the relatively **high branching factor**, the requirement for **low per-decision computation time**, and the **uncertainty introduced by non-deterministic actions**.

The algorithm proceeds through the four canonical phases: **selection, expansion, simulation, and backpropagation**.

During **selection**, the algorithm traverses the tree from the root to a leaf by choosing actions according to the **Upper Confidence Bound (UCB)** formula

In the **expansion** phase, when the algorithm encounters a state with unexplored actions, it adds a **single** new node to the tree corresponding to one such action.

**Simulation** then takes over from the newly added node, playing out the rest of the turn randomly until the move that ends the turn is reached. This outcome provides a utility value $u \in \mathbb{R}$.

In the **backpropagation** phase, the utility $u$ is propagated back up the tree, updating the statistics of all nodes along the path taken during selection. Specifically, we update the **visit counts**, as well as the **maximum** and the **mean** of the utility values.

The **non-determinism** is explicitly handled by or MCTS implementation. At each traversal of the search tree, instead of relying solely on the children generated during the expansion phase, these are compared with a newly generated set of possible successors. If a state appears during traversal that was not present in the initially expanded set, it is included, as it represents a possible outcome arising from the game’s inherent randomness. This ensures that the search accounts for multiple stochastic outcomes rather than being constrained to a single deterministic path.

#### Applying Prior Moves and Prior Choices
To reduce both the branching factor and the depth explored by MCTS, we heuristically execute moves that are consistently advantageous in their context, there are two types of such moves.
- **Prior moves** are a set of actions that, when **all** are played, produce the same outcome regardless of order. This set includes actions or cards that grant coins, prestige, or power. Since increases in these resources are generally advantageous and the order of these moves doesn’t affect combo activation, the set can be executed immediately in any order.
- **Prior choices** are a set of actions where selections must be made from a set of options. Each choice is made **greedily**: options are evaluated using a *utility function* that estimates the resulting game state's value, and the highest-scoring option is selected at each step without considering the globally optimal order.


#### Bot Design

The goal of the agent is to **maximize a utility function** while completing its turn within a strict time limit $T$ (in milliseconds). To achieve this, we follow the following scheme:
1. Apply all **Prior Moves** and **Prior Choices** to reduce the number of decisions that MCTS needs to consider.
2. Use MCTS to select a move from the remaining options within the remaining time.
3. GOTO 1. until the turn is finished or the time limit is reached.

This approach ensures that we do not waste time on moves that are consistently good, allowing us to focus the time budget on more complex and critical decisions.

To ensure that MCTS move selection stays within the total time $T$, we impose two constraints: a maximum number of iterations $N_{\max}$ witch is an hyperparameter, and a **time budget**.

To allocate the **time budget** $t$ effectively, we want to spend more time on the first choice, since it corresponds to a larger subtree and requires more exploration to select the best move. To achieve this, the algorithm estimates the branching factor at each depth as $b = [b_0, b_1, \dots, b_{d}]$ by performing random playouts for each possible move.

The total time $T$ can be expressed as the sum of the time spent at each depth:

$$
T = \sum_{i=0}^{d} t_i
$$

with

$$
t_i = b_i \cdot t_{i+1}
\;\;\iff\;\;
t_{i+1} = \cfrac{t_i}{b_i}
\;\;\iff\;\;
t_i = \cfrac{t_0}{\prod_{j=0}^{i-1} b_j}.
$$

Therefore, $t_0$ can be determined by scaling the total time $T$ according to the estimated size of the tree:

$$
t_0 = \cfrac{T}{1 + \cfrac{1}{b_0} + \cfrac{1}{b_0 b_1} + \cfrac{1}{b_0 b_1 b_2} + \dots + \cfrac{1}{b_0 b_1 \dots b_{d-1}}}.
$$

Doing so for each possible move, we obtain a list of candidate times $t_0(i)$, where $i$ indicates a move.
To account for the computation time spent during this calculation, the **time budget** is allocated as:

$$
t =  \min_i \big(t_0(i)\big), - \text{TimeSpent}.
$$

#### Heuristic Design: Min–Max Hand Value Rating with Weighted Evaluation

The proposed utility function evaluates a given game state by combining domain-specific features into a single utility score. It relies on a feature vector that summarizes the player’s current standing in terms of both resources and long-term potential as represented by the quality of the deck.

the component features of the  heuristic include:
- **Prestige and Power**: direct indicators of victory progress.
- **Coins Remaining**: current coin reserves, reflecting short-term purchasing capability.
- **Patron Favor**: net advantage in patron allegiance, computed as the difference between the number of patrons favoring the player and those aligned with the opponent.
- **Opponent Prestige**: a measure of the adversary’s relative progress.

the other are two instances of the **Min–Max Hand Value Rating (MMHVR)** model, which evaluates the quality of the player’s deck with respect to specific resources. It is applied separately to:
1. **Coin-related card effects**: representing economic potential.
2. **Prestige/Power-related card effects**: representing victory-point potential.

For each category, MMHVR computes the following statistics:
- **Top Hand Value**: sum of the five highest individual card gains.
- **Bottom Hand Value**: sum of the five lowest individual card gains.
- **Average Hand Value**: mean of the top and bottom hand values.
- **Average Single Card Value**: mean gain per card across the deck.

The resulting representation is a **13-dimensional feature vector** $\mathbf{x} = [x_0,\dots,x_{12}]$

The heuristic comes in two flavors. The **plain version** uses the raw feature values directly defined as: $$U_{\text{plain}}(\mathbf{x}) = \sum_{i=1}^{n} x_i$$ and the **engineered version** applies nonlinear transformations to selected features in order to better capture their strategic meaning in each phase of the game and so is defined as $$U_{\text{ing}}(\mathbf{x}) = \sum_{i=1}^{n} f_i(x_i)$$ For example, top-hand coin values are logarithmically scaled to model diminishing returns, prestige and power potential are exponentially weighted to emphasize late-game importance, patron favor is squared with sign preservation to reward decisive control, and negative weights are assigned to remaining coins and opponent prestige to penalize excessive hoarding and strong adversary positions. This formulation embeds domain knowledge into the evaluation, encouraging the search process to favor strategically aligned states.

Both versions can also operate in a **weighted mode**, where each feature is multiplied by a weight vector $\mathbf{w}$ and optionally transformed by activation functions:
$$U_{\text{weighted}}(\mathbf{x},\mathbf{w}) = \sum_{i=1}^{n} f_i(w_i x_i)$$
This enables adaptation of the heuristic to different strategic styles, either through manual tuning or automated optimization.

A post-processing adjustment accounts for terminal outcomes: the utility is multiplied by 3 in the case of a win, while it is halved in the case of a loss.

#### Evolutionary Optimization of Weights and Activation Functions

The proposed approach employs an evolutionary algorithm to simultaneously optimize both the **weights** and the **activation functions** used in the **MMHVR weighted utility function**. Unlike conventional methods that focus solely on adapting real-valued weights, our method also considers activation function selection as part of the optimization process. This dual encoding allows the algorithm to explore not only adjustments of the parameters but also changes in the functional form of the responses. The goal is to capture how the importance of each feature in the heuristic may shift as the game progresses.

Each candidate solution, or *individual*, is represented by a pair of vectors: a continuous vector of real-valued weights and a discrete vector of activation function identifiers. The activation functions are drawn from a predefined set, which constrains the search space.

The evolutionary process begins with the initialization of a population of individuals. Weights are sampled from a standard normal distribution, promoting variability in the initial behaviors, while activation functions are assigned uniformly at random.

Fitness evaluation is performed through direct competition. Each individual engages in a series of matches against all other members of the population, alternating between the roles of first and second player to account for the inherent advantage associated with moving first. Each individual competes at least once in both roles against every opponent. Fitness is quantified as the ratio of games won to the total number of games played.

The agents are bounded-depth search agents (depth limit = 2) that, at each decision step, select the move leading to the state that maximizes the **Weighted MMHVR utility function**, using the **Prior Move** and **Prior Choice** heuristics described earlier to enhance computational efficiency. Each agent is parameterized by its specific weights and activations used in computing the **Weighted MMHVR utility function**. This simple search strategy was chosen to allow the heuristic to be trained independently of more sophisticated algorithms.

Parent selection relies on **tournament selection**. In each tournament, a subset of individuals is sampled without replacement, and the one with the highest fitness is chosen as a parent. The process is repeated to obtain the second parent.

New offspring are created through a combination of crossover and mutation. Crossover is implemented as single-point recombination applied simultaneously to both the weight and activation vectors. Mutation perturbs weights with Gaussian noise, while activation functions are occasionally replaced with randomly chosen alternatives.

To preserve high-performing solutions, the algorithm also employs **elitism**, carrying over a small number of the best individuals unchanged into the next generation. This prevents the loss of the strongest strategies due to the randomness of genetic operations.

Given that training can be lengthy and computationally demanding, a checkpointing mechanism was implemented: at the end of each generation, the full population is saved to disk, allowing the process to be resumed after potential interruptions.

To assess the progression of the algorithm, convergence metrics are derived from the population. These include the mean and standard deviation of the weights, as well as the diversity of activation functions across parameters. An example of the recorded convergence data is shown in Figure:
![Convergence Metric](img/ConvergenceMetric.png)

We can observe a transition in the heat map when the mutation rate of the activation function is reduced.

### Assessment

The assessment is done by comparing the performance of different bot configurations.

**Bots used:**
- **AIFBotMCTS**: our bot described earlier that uses MCTS with a specified evaluation function as a parameter.
- **BoundedDS**: a bounded-depth search bot that uses a specified evaluation function to evaluate the state and pick the move that leads to the best state after a fixed depth.
- **RandomBot**: selects moves uniformly at random.

**Heuristics considered:**
- **WeightedUtilityFunction_MMHVR**: the engineered version of the MMHVR heuristic.
- **WeightedUtilityFunction_MMHVR_plain**: the plain version of the MMHVR heuristic.
- **utilityFunction_PrestigeAndPower**: a simple heuristic that sums the prestige and power of the player.


#### Experiment

Here is a list of the initial configurations required to run all experiments.

In [ ]:
from ExampleBot.RandomBot import RandomBot
from HeuristicLearning.EvolutionaryHeuristic  import Individual
from bots.AIFBot_MCTS import AIFBotMCTS, MCTSenum
from bots.BoundedDS import BoundedDS

from BotCommon.Heuristics import WeightedUtilityFunction_MMHVR, WeightedUtilityFunction_MMHVR_plain, utilityFunction_PrestigeAndPower
from Helper.GameManager import TryAsFirstAndSecondPlayer

HIDE_PRINT = False
RUN_NUM = 100

THREAD_NUM = 5

DEPTH = 2

this experiment aims to establish whether our heuristic is effective by testing if it can outperform a bot that plays randomly.

In [ ]:
bot_random           = RandomBot(bot_name="RandomBot")
bot_BoundedDS_WMMHVR = BoundedDS(bot_name="BDS_WMMHVR", depth=DEPTH,evaluation_function=WeightedUtilityFunction_MMHVR)

TryAsFirstAndSecondPlayer(bot_BoundedDS_WMMHVR, bot_random, runs=RUN_NUM, threads=THREAD_NUM, hide_print=HIDE_PRINT)

In [ ]:
# PLAYER 1: BDS_WMMHVR | PLAYER 2 : RandomBot
# Stats from the games played:
# Final amount of draws: 0/100 (0%)
# Final amount of P1 wins: 100/100 (100%)
# Final amount of P2 wins: 0/100 (0%)
# Ends due to Prestige>40: 65/100 (65%)
# Ends due to Prestige>80: 0/100 (0%)
# Ends due to Patron Favor: 35/100 (35%)
# Ends due to Turn Limit: 0/100 (0%)
# Ends due to other factors: 0/100 (0%)
#
# PLAYER 1: RandomBot | PLAYER 2 : BDS_WMMHVR
# Stats from the games played:
# Final amount of draws: 0/100 (0%)
# Final amount of P1 wins: 0/100 (0%)
# Final amount of P2 wins: 100/100 (100%)
# Ends due to Prestige>40: 75/100 (75%)
# Ends due to Prestige>80: 0/100 (0%)
# Ends due to Patron Favor: 25/100 (25%)
# Ends due to Turn Limit: 0/100 (0%)
# Ends due to other factors: 0/100 (0%)

This experiment serves as a baseline for the heuristic and tests whether it can outperform a simpler, effective heuristic, namely **Max Prestige**, which simply sums the prestige and power of a given state.
As the simplest heuristic that produces meaningful results, it is used as a secondary baseline.

In [ ]:
bot_BoundedDS_MAXPRESTIGE = BoundedDS(bot_name="BDS_MAXPRESTIGE", depth=DEPTH,evaluation_function=utilityFunction_PrestigeAndPower)
bot_BoundedDS_WMMHVR = BoundedDS(bot_name="BDS_WMMHVR", depth=DEPTH,evaluation_function=WeightedUtilityFunction_MMHVR)

TryAsFirstAndSecondPlayer(bot_BoundedDS_WMMHVR, bot_BoundedDS_MAXPRESTIGE, runs=RUN_NUM, threads=THREAD_NUM, hide_print=HIDE_PRINT)

In [ ]:
# PLAYER 1: BDS_WMMHVR | PLAYER 2 : BDS_MAXPRESTIGE
# Stats from the games played:
# Final amount of draws: 0/100 (0%)
# Final amount of P1 wins: 61/100 (61%)
# Final amount of P2 wins: 39/100 (39%)
# Ends due to Prestige>40: 95/100 (95%)
# Ends due to Prestige>80: 0/100 (0%)
# Ends due to Patron Favor: 5/100 (5%)
# Ends due to Turn Limit: 0/100 (0%)
# Ends due to other factors: 0/100 (0%)
#
# PLAYER 1: BDS_MAXPRESTIGE | PLAYER 2 : BDS_WMMHVR
# Stats from the games played:
# Final amount of draws: 0/100 (0%)
# Final amount of P1 wins: 51/100 (51%)
# Final amount of P2 wins: 49/100 (49%)
# Ends due to Prestige>40: 93/100 (93%)
# Ends due to Prestige>80: 2/100 (2%)
# Ends due to Patron Favor: 5/100 (5%)
# Ends due to Turn Limit: 0/100 (0%)
# Ends due to other factors: 0/100 (0%)

This experiment compares the plain heuristic with a manually engineered version, both described earlier, to assess whether the engineered heuristic provides any improvement.

In [ ]:
bot_BoundedDS_WMMHVR_P = BoundedDS(bot_name="BDS_MMHVR_Plain", depth=DEPTH,evaluation_function=WeightedUtilityFunction_MMHVR_plain)
bot_BoundedDS_WMMHVR = BoundedDS(bot_name="BDS_WMMHVR", depth=DEPTH,evaluation_function=WeightedUtilityFunction_MMHVR)

TryAsFirstAndSecondPlayer(bot_BoundedDS_WMMHVR_P, bot_BoundedDS_WMMHVR, runs=RUN_NUM, threads=THREAD_NUM, hide_print=HIDE_PRINT)

In [ ]:
# PLAYER 1: BDS_MMHVR_Plain | PLAYER 2 : BDS_WMMHVR
# Stats from the games played:
# Final amount of draws: 0/100 (0%)
# Final amount of P1 wins: 44/100 (44%)
# Final amount of P2 wins: 56/100 (56%)
# Ends due to Prestige>40: 73/100 (73%)
# Ends due to Prestige>80: 0/100 (0%)
# Ends due to Patron Favor: 27/100 (27%)
# Ends due to Turn Limit: 0/100 (0%)
# Ends due to other factors: 0/100 (0%)
#
# PLAYER 1: BDS_WMMHVR | PLAYER 2 : BDS_MMHVR_Plain
# Stats from the games played:
# Final amount of draws: 0/100 (0%)
# Final amount of P1 wins: 74/100 (74%)
# Final amount of P2 wins: 26/100 (26%)
# Ends due to Prestige>40: 71/100 (71%)
# Ends due to Prestige>80: 0/100 (0%)
# Ends due to Patron Favor: 29/100 (29%)
# Ends due to Turn Limit: 0/100 (0%)
# Ends due to other factors: 0/100 (0%)


This experiment assesses the correctness of our Monte Carlo implementation by testing whether it can outperform the random bot.

In [ ]:
bot_MCTS_MAXPRESTIGE = AIFBotMCTS(bot_name="MCTS_MAXPRESTIGE", evaluation_function=utilityFunction_PrestigeAndPower, MCTSversion=MCTSenum.MCTS2)
bot_random           = RandomBot(bot_name="RandomBot")

TryAsFirstAndSecondPlayer(bot_random, bot_MCTS_MAXPRESTIGE, runs=RUN_NUM, threads=THREAD_NUM, hide_print=HIDE_PRINT)

In [ ]:
# PLAYER 1: RandomBot | PLAYER 2 : MCTS_MAXPRESTIGE
# Stats from the games played:
# Final amount of draws: 0/100 (0%)
# Final amount of P1 wins: 0/100 (0%)
# Final amount of P2 wins: 100/100 (100%)
# Ends due to Prestige>40: 100/100 (100%)
# Ends due to Prestige>80: 0/100 (0%)
# Ends due to Patron Favor: 0/100 (0%)
# Ends due to Turn Limit: 0/100 (0%)
# Ends due to other factors: 0/100 (0%)
#
# PLAYER 1: MCTS_MAXPRESTIGE | PLAYER 2 : RandomBot
# Stats from the games played:
# Final amount of draws: 0/100 (0%)
# Final amount of P1 wins: 100/100 (100%)
# Final amount of P2 wins: 0/100 (0%)
# Ends due to Prestige>40: 100/100 (100%)
# Ends due to Prestige>80: 0/100 (0%)
# Ends due to Patron Favor: 0/100 (0%)
# Ends due to Turn Limit: 0/100 (0%)
# Ends due to other factors: 0/100 (0%)


This experiment evaluates the performance of the heuristic within the Monte Carlo Tree Search by comparing a bot using the **MMHVR** heuristic with one using the **Max Prestige** heuristic.

In [ ]:
bot_MCTS_MMHVR = AIFBotMCTS(bot_name="MCTS_MHHVR", evaluation_function=WeightedUtilityFunction_MMHVR, MCTSversion=MCTSenum.MCTS2)
bot_MCTS_MAXPRESTIGE = AIFBotMCTS(bot_name="MCTS_MAXPRESTIGE", evaluation_function=utilityFunction_PrestigeAndPower, MCTSversion=MCTSenum.MCTS2)

TryAsFirstAndSecondPlayer(bot_MCTS_MMHVR, bot_MCTS_MAXPRESTIGE, runs=RUN_NUM, threads=THREAD_NUM, hide_print=HIDE_PRINT)

In [ ]:
# PLAYER 1: MCTS_MHHVR | PLAYER 2 : MCTS_MAXPRESTIGE
# Stats from the games played:
# Final amount of draws: 0/100 (0%)
# Final amount of P1 wins: 77/100 (77%)
# Final amount of P2 wins: 23/100 (23%)
# Ends due to Prestige>40: 93/100 (93%)
# Ends due to Prestige>80: 0/100 (0%)
# Ends due to Patron Favor: 7/100 (7%)
# Ends due to Turn Limit: 0/100 (0%)
# Ends due to other factors: 0/100 (0%)
#
# PLAYER 1: MCTS_MAXPRESTIGE | PLAYER 2 : MCTS_MHHVR
# Stats from the games played:
# Final amount of draws: 0/100 (0%)
# Final amount of P1 wins: 38/100 (38%)
# Final amount of P2 wins: 62/100 (62%)
# Ends due to Prestige>40: 94/100 (94%)
# Ends due to Prestige>80: 0/100 (0%)
# Ends due to Patron Favor: 6/100 (6%)
# Ends due to Turn Limit: 0/100 (0%)
# Ends due to other factors: 0/100 (0%)

The next experiment compares the **plain** heuristic with the version whose weights were optimized using an evolutionary algorithm, to assess whether the evolutionary approach can yield improvements.

In [ ]:
ind = Individual.LoadLatest("gen")
bot_MCTS_WMMHVR_evolved   = AIFBotMCTS(bot_name="MCTS_MHHVR_evolved", evaluation_function=WeightedUtilityFunction_MMHVR_plain,
                                       weights=ind.weights, functions=ind.activations, MCTSversion=MCTSenum.MCTS2)
bot_MCTS_WMMHVR_P         = AIFBotMCTS(bot_name="MCTS_MHHVR_P",
                                       evaluation_function= WeightedUtilityFunction_MMHVR_plain, MCTSversion=MCTSenum.MCTS2)

TryAsFirstAndSecondPlayer(bot_MCTS_WMMHVR_evolved, bot_MCTS_WMMHVR_P, runs=RUN_NUM, threads=THREAD_NUM, hide_print=HIDE_PRINT)

In [ ]:
# PLAYER 1: MCTS_MHHVR_evolved | PLAYER 2 : MCTS_MHHVR_P
# Stats from the games played:
# Final amount of draws: 0/100 (0%)
# Final amount of P1 wins: 81/100 (81%)
# Final amount of P2 wins: 19/100 (19%)
# Ends due to Prestige>40: 71/100 (71%)
# Ends due to Prestige>80: 0/100 (0%)
# Ends due to Patron Favor: 29/100 (29%)
# Ends due to Turn Limit: 0/100 (0%)
# Ends due to other factors: 0/100 (0%)
#
# PLAYER 1: MCTS_MHHVR_P | PLAYER 2 : MCTS_MHHVR_evolved
# Stats from the games played:
# Final amount of draws: 0/100 (0%)
# Final amount of P1 wins: 20/100 (20%)
# Final amount of P2 wins: 80/100 (80%)
# Ends due to Prestige>40: 72/100 (72%)
# Ends due to Prestige>80: 0/100 (0%)
# Ends due to Patron Favor: 28/100 (28%)
# Ends due to Turn Limit: 0/100 (0%)
# Ends due to other factors: 0/100 (0%)


The next experiment compares the **engineered** heuristic with the version whose weights were optimized using an evolutionary algorithm, to assess whether the evolutionary approach can yield improvements.

In [ ]:
ind = Individual.LoadLatest("gen")
bot_MCTS_WMMHVR_evolved   = AIFBotMCTS(bot_name="MCTS_MHHVR_evolved", evaluation_function=WeightedUtilityFunction_MMHVR_plain,
                                       weights=ind.weights, functions=ind.activations, MCTSversion=MCTSenum.MCTS2)
bot_MCTS_WMMHVR_ing     = AIFBotMCTS(bot_name="MCTS_MHHVR_ing",evaluation_function= WeightedUtilityFunction_MMHVR, MCTSversion=MCTSenum.MCTS2)

TryAsFirstAndSecondPlayer(bot_MCTS_WMMHVR_evolved, bot_MCTS_WMMHVR_ing, runs=RUN_NUM, threads=THREAD_NUM, hide_print=HIDE_PRINT)

In [ ]:
# PLAYER 1: MCTS_MHHVR_evolved | PLAYER 2 : MCTS_MHHVR_ing
# Stats from the games played:
# Final amount of draws: 0/100 (0%)
# Final amount of P1 wins: 52/100 (52%)
# Final amount of P2 wins: 48/100 (48%)
# Ends due to Prestige>40: 96/100 (96%)
# Ends due to Prestige>80: 0/100 (0%)
# Ends due to Patron Favor: 4/100 (4%)
# Ends due to Turn Limit: 0/100 (0%)
# Ends due to other factors: 0/100 (0%)
#
# PLAYER 1: MCTS_MHHVR_ing | PLAYER 2 : MCTS_MHHVR_evolved
# Stats from the games played:
# Final amount of draws: 0/100 (0%)
# Final amount of P1 wins: 74/100 (74%)
# Final amount of P2 wins: 26/100 (26%)
# Ends due to Prestige>40: 95/100 (95%)
# Ends due to Prestige>80: 0/100 (0%)
# Ends due to Patron Favor: 5/100 (5%)
# Ends due to Turn Limit: 0/100 (0%)
# Ends due to other factors: 0/100 (0%)


This experiment evaluates the performance of the Monte Carlo Tree Search bot by comparing it against a simpler bot that uses the same heuristic.

In [ ]:
bot_MCTS_MMHVR = AIFBotMCTS(bot_name="MCTS_MHHVR", evaluation_function=WeightedUtilityFunction_MMHVR, MCTSversion=MCTSenum.MCTS2)
bot_BoundedDS_WMMHVR = BoundedDS(bot_name="BDS_WMMHVR", depth=DEPTH,evaluation_function=WeightedUtilityFunction_MMHVR)

TryAsFirstAndSecondPlayer(bot_MCTS_MMHVR, bot_BoundedDS_WMMHVR, runs=RUN_NUM, threads=THREAD_NUM, hide_print=HIDE_PRINT)

In [ ]:
# PLAYER 1: MCTS_MHHVR | PLAYER 2 : BDS_WMMHVR
# Stats from the games played:
# Final amount of draws: 0/100 (0%)
# Final amount of P1 wins: 87/100 (87%)
# Final amount of P2 wins: 13/100 (13%)
# Ends due to Prestige>40: 93/100 (93%)
# Ends due to Prestige>80: 0/100 (0%)
# Ends due to Patron Favor: 7/100 (7%)
# Ends due to Turn Limit: 0/100 (0%)
# Ends due to other factors: 0/100 (0%)
#
# PLAYER 1: BDS_WMMHVR | PLAYER 2 : MCTS_MHHVR
# Stats from the games played:
# Final amount of draws: 0/100 (0%)
# Final amount of P1 wins: 29/100 (29%)
# Final amount of P2 wins: 71/100 (71%)
# Ends due to Prestige>40: 96/100 (96%)
# Ends due to Prestige>80: 0/100 (0%)
# Ends due to Patron Favor: 4/100 (4%)
# Ends due to Turn Limit: 0/100 (0%)
# Ends due to other factors: 0/100 (0%)

In this setup, Prior Moves are also applied to the simpler bot to evaluate their impact.

In [ ]:
bounded_prior = BoundedDS(bot_name="bounded_prior", depth=DEPTH, evaluation_function=WeightedUtilityFunction_MMHVR,
                          use_prior_move=True)
bot_MCTS_MMHVR = AIFBotMCTS(bot_name="MCTS_MHHVR", evaluation_function=WeightedUtilityFunction_MMHVR, MCTSversion=MCTSenum.MCTS2)
TryAsFirstAndSecondPlayer(bot_MCTS_MMHVR, bounded_prior, runs=RUN_NUM, threads=THREAD_NUM, hide_print=HIDE_PRINT)

In [ ]:
# PLAYER 1: MCTS_MHHVR | PLAYER 2 : bounded_prior
# Stats from the games played:
# Final amount of draws: 0/100 (0%)
# Final amount of P1 wins: 58/100 (58%)
# Final amount of P2 wins: 42/100 (42%)
# Ends due to Prestige>40: 94/100 (94%)
# Ends due to Prestige>80: 1/100 (1%)
# Ends due to Patron Favor: 5/100 (5%)
# Ends due to Turn Limit: 0/100 (0%)
# Ends due to other factors: 0/100 (0%)
#
# PLAYER 1: bounded_prior | PLAYER 2 : MCTS_MHHVR
# Stats from the games played:
# Final amount of draws: 0/100 (0%)
# Final amount of P1 wins: 62/100 (62%)
# Final amount of P2 wins: 38/100 (38%)
# Ends due to Prestige>40: 95/100 (95%)
# Ends due to Prestige>80: 0/100 (0%)
# Ends due to Patron Favor: 5/100 (5%)
# Ends due to Turn Limit: 0/100 (0%)
# Ends due to other factors: 0/100 (0%)

### Conclusion

The experiments show that:
- **Heuristics provide clear advantages**: MMHVR outperforms random play and is slightly stronger than MaxPrestige, confirming its effectiveness.
- **Engineered vs. plain heuristics**: manual tuning improves performance, while evolutionary weights outperform plain versions but remain close to the engineered heuristic.
- **MCTS validation**: MCTS with even simple heuristics dominates random play, confirming correct implementation.
- **Heuristic strength in MCTS**: stronger heuristics (MMHVR) achieve significantly higher win rates under MCTS compared to simpler ones.
- **MCTS vs. bounded search**: MCTS with prior moves consistently outperforms bounded-depth search. However, when prior moves are also enabled in bounded search, it achieves a higher win rate than MCTS with prior moves. We hypothesize that this occurs because the heuristic assigns only slightly higher values to better states. Bounded search, by selecting the maximum utility, is less sensitive to this kind of noise compared to MCTS. Nonetheless, this remains only a hypothesis.


### Bibliografy
- [A Survey of Monte Carlo Tree Search Methods](https://ieeexplore.ieee.org/abstract/document/6145622)
- [Developing Card Playing Agent forTales of Tribute AI Competition](https://jakubkowalski.tech/Supervising/Ciezkowski2023DevelopingCard.pdf)
- [Fullstack Academy - Monte Carlo Tree Search (MCTS) Tutorial](https://www.youtube.com/watch?v=Fbs4lnGLS8M)
